# Arithmetic Method AGL

This is a SageMath script that computes the motive of the $\mathrm{AGL}_1(k)$-representation variety over a compact orientable surface of genus $g$, $\Sigma_g$. That is, this is the space of representations
$$
    \rho: \pi_1(\Sigma_g) \to \mathrm{AGL}_1(k).
$$

The computation is performed by counting the number of points of the representation variety over the finite field of $q$ elements, $\mathbb{F}_q$. The calculation is based on the result of Katz in [2]. Roughly speaking, this result shows that, given a $\mathbb{Z}$-scheme $X$, if the function $P(q) = \#X(\mathbb{F}_q)$ is a polynomial in $q$, then $P(uv) = E(X(\mathbb{C}))$ is the $E$-polynomial of $X(\mathbb{C})$.

From this calculation for small genus, the existence of a Topological Quantum Field Theory computing the $E$-polynomials allows us to compute the $E$-polynomial for arbitrary genus. For further information about the method, please check [1].

**Usage.** For computing the $E$-polynomials up to genus $g$ (in this example, say up to $g = 3$) just type

    compute_polynomials(3)

**Authors:** Ángel González-Prieto (Universidad Politécnica de Madrid), Marina Logares (Universidad Complutense de Madrid) and Vicente Muñoz (Universidad de Málaga).

### References.

[1] Á. González-Prieto, M. Logares and V. Muñoz, *Representation variety for the rank one affine group*, arXiv:2005.01841.

[2] T. Hausel and F. Rodrı́guez-Villegas, *Mixed Hodge polynomials of character varieties*. With an appendix by
Nicholas M. Katz, Invent. Math., 174 (2008), 555–624.

In [1]:
%load_ext cython
from sage.all import *

In [ ]:
%%cython
from sage.all import *

'''
    Generates the possible values of beta
    within the field k.
'''
def generate_beta(s, k):
    if s == 0:
        return [[]]
    else:
        betas = generate_beta(s-1, k)
        result = []
        for beta_tuple in betas:
            for beta in k:
                result.append(beta_tuple + [beta])
    return result

'''
    Generates the possible values of alpha
    within the field k.
'''
def generate_alpha(s, k):
    if s == 0:
        return [[]]
    else:
        alphas = generate_alpha(s-1, k)
        result = []
        for alpha_tuple in alphas:
            for alpha in k:
                if alpha != -1: # -1 is not allowed for alpha
                    result.append(alpha_tuple + [alpha])
    return result

'''
    Counts the number of points of the
    AGL(1, F_q)-representation variety
    of the compact orientable surface
    of genus g.
    Here, F_q is the finite field of
    q elements.
'''
cdef count_points(g, q):
    k = GF(q)
    cdef list betas = generate_beta(2*g, k)
    cdef list alphas = generate_alpha(2*g, k)
    
    cdef int n_points = 0
    cdef int counter = int(0)
    
    # Shows a message each 10% of computation
    step_msg = ceil(len(betas)/10)
    for beta in betas:
        if step_msg > 0 and counter % step_msg == 0:
            print('--> ' + str(counter / step_msg * 10) + '%')
        counter += 1
        
        for alpha in alphas:
            # Checks the equation defining
            # the representation variety
            if sum([alpha[i]*beta[i] for i in range(2*g)]) == 0:
                n_points += 1
    return n_points

'''
    Computes the values of the E-polynomial
    (the number of points) of the representation
    variety over the surface of genus g
    up to degree d.
'''
def compute_values(g, d):
    values = []
    n_values = 0
    q = 1
    while n_values < d + 1:
        q += 1
        if is_prime_power(q):
            print('Computing for g = ' + str(g)\
                  + ' and q = '+str(q))
            n_points = count_points(g, q)
            values.append([q, n_points])
            n_values += 1
    
    return values

'''
    Computes the E-polynomial of the
    AGL(1,C)-representation variety over
    the genus g surface.
    This function requieres an upper bound,
    deg_estimate, of the degree of the
    E-polynomial.
'''
def compute_polynomial(g, deg_estimate):
    d = deg_estimate(g)
    values = compute_values(g, d)
    R = PolynomialRing(QQ, 'q')
    P_counted = R.lagrange_polynomial(values)
    return P_counted

'''
    TOP FUNCTION
    Computes the E-polynomial of the
    AGL(1,C)-representation variety over
    the genus g surface for g = 1, 2, ..., g_max.
    This function requieres a function,
    deg_estimate, that returns an upper bound
    of the E-polynomial for each degree.
    Returns a list with the E-polynomials
    ordered by genus.
'''
def compute_polynomials(g_max):
    result = [0]*(g_max+1)
    deg_estimate = lambda g: 4*g -1
    for g in range(1, g_max+1):
        result[g] = compute_polynomial(g, deg_estimate)
        print('+++++++++++++++++')
        show('\\textrm{Result for }g = ' + str(g) + ': ' + latex(result[g]))
        print('+++++++++++++++++')
        
    return result